In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from dtw import dtw
from dtw import accelerated_dtw
from scipy.spatial.distance import cosine
from fastdtw import fastdtw
import itertools
from  numpy.linalg import norm
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
# 查看路径
dir = "/home/liu/桌面"  #根据自己目录修改
print(os.listdir(dir))
timeseries = pd.read_csv("/home/liu/files/files/repo/dtw/df_all.csv",index_col=0)
timeseries = timeseries.fillna(0)
euclidean_norm = lambda x, y: np.abs(x - y)/(np.abs(x)+np.abs(y))    
# euclidean_norm = lambda x, y: np.abs(x - y)

['p1285-duA.pdf', '.ipynb_checkpoints', 'linux+实验报告模板2.doc', '1873578351.jpg']


In [3]:
#计算相应的关系系数矩阵

corr_data=pd.DataFrame(columns=timeseries.columns[:5],index=timeseries.columns[:5])
#传入时序数据并且将其标准化之后返回
def scale(timeseries):
    scale=StandardScaler()
    for i in timeseries.columns:
        timeseries[i]=scale.fit_transform(timeseries[i].values.reshape(-1,1))
    return timeseries

#将标准化之后的矩阵计算相关系数矩阵
def corrdata_fun(timeseries_scale):
    for i in timeseries_scale.columns:
        for j in timeseries_scale.columns:
            distance,path=fastdtw(timeseries_scale[i],timeseries_scale[j],dist=euclidean_norm)
            distance=distance/(2*len(timeseries_scale))
            corr_data.loc[i,j]=distance
            corr_data.loc[j,i]=distance
    return corr_data
timeseries_scale=scale(timeseries.iloc[:,:5])
corr_data=corrdata_fun(timeseries_scale)
print(corr_data)

/home/liu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


                 万能打印_count 生成万能当前打印任务_0 生成万能当前打印任务_3 生成万能当前打印任务_4  \
万能打印_count                0     0.480368     0.432457      0.41065   
生成万能当前打印任务_0       0.480368            0     0.115571     0.173515   
生成万能当前打印任务_3       0.432457     0.115571            0    0.0746558   
生成万能当前打印任务_4        0.41065     0.173515    0.0746558            0   
生成万能当前打印任务_count   0.373934     0.240318      0.15307    0.0888295   

                 生成万能当前打印任务_count  
万能打印_count               0.373934  
生成万能当前打印任务_0             0.240318  
生成万能当前打印任务_3              0.15307  
生成万能当前打印任务_4            0.0888295  
生成万能当前打印任务_count                0  


In [4]:
%%time
def time_num(time):
    hour=time.split(" ")[1].split(":")[0]
    minu=time.split(" ")[1].split(":")[1]
    num=int(minu)+(int(hour)*60)
    return num

def num_time(num):
    hour=num//60
    min=num - hour * 60
    return str(hour)+":"+str(min)

#corr_data表示相关性矩阵
#timeseries_scale表示经过标准化之后的矩阵
#type表示属性种类
def order(type_,num,corr_data,timeseries_scale):
    order=list(corr_data.loc[type_].sort_values().index) 
    result=pd.DataFrame(index=order,columns=["Time"])                   #result:最终的时间dataframe
    for i in order:
        x = timeseries_scale[type_].values.reshape(-1,1)
        y = timeseries_scale[i].values.reshape(-1,1)
        p1,b1 = fastdtw(x,y,dist=euclidean_norm)
        b1=pd.DataFrame(b1)                          #b1是得到的路径DataFrame
        the_result=b1[1][num]                        #the_result表示得到的时间点
        result.loc[i,"Time"]=the_result
    return result


def influence(type,time,corr_data,timeseries_scale):
    num = time_num(time)
    all_influ=order(type,num,corr_data,timeseries_scale)
    all_influ=all_influ.drop(index=type)
    return all_influ['Time'].apply(num_time)


all_influ=influence("万能打印_count","2018-06-01 23:09",corr_data,timeseries_scale)

CPU times: user 14.7 s, sys: 24 ms, total: 14.7 s
Wall time: 14.7 s


In [6]:
all_influ

生成万能当前打印任务_count    17:12
生成万能当前打印任务_4        11:29
生成万能当前打印任务_3        17:52
生成万能当前打印任务_0        12:28
Name: Time, dtype: object